In [1]:
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision
import numpy as np
import random

import torch
import torch.nn.functional as F
import cl_gym as cl

import sys
import os

seed = 1

np.random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.enabled = False

def make_params() -> dict:
    import os
    from pathlib import Path
    import uuid

    params = {
            # dataset
            'dataset': "MNIST",
            # 'dataset': "FMNIST",

            # benchmark
            'seed': seed,
            'num_tasks': 5,
            'epochs_per_task': 1,
            # 'per_task_examples': np.inf,
            'per_task_examples': 10000,
            'per_task_memory_examples': 20,
            'batch_size_train': 64,
            'batch_size_memory': 64,
            'batch_size_validation': 256,
            'tau': 10,
            # 'tau': 0.0,

            # algorithm
            'optimizer': 'sgd',
            'learning_rate': 0.001,
            'momentum': 0.9,
            'learning_rate_decay': 1.0,
            'criterion': torch.nn.CrossEntropyLoss(),
            'device': torch.device('cuda:7' if torch.cuda.is_available() else 'cpu'),
             
            # sample selection
            'alpha':0.001
              }

#     trial_id = str(uuid.uuid4())
    trial_id = f"demo/dataset={params['dataset']}/seed={params['seed']}_epoch={params['epochs_per_task']}_lr={params['learning_rate']}_alpha={params['alpha']}_tau={params['tau']}"
    params['trial_id'] = trial_id
    params['output_dir'] = os.path.join("./outputs/{}".format(trial_id))
    print(f"output_dir={params['output_dir']}")
    Path(params['output_dir']).mkdir(parents=True, exist_ok=True)

    return params

params = make_params()

output_dir=./outputs/dataset=MNIST/seed=1_epoch=1_lr=0.001_alpha=0.001_tau=10


In [2]:
from datasets import MNIST
from datasets import FashionMNIST
from datasets import CIFAR10, CIFAR100

if params['dataset'] == 'MNIST':
    benchmark = MNIST(num_tasks=params['num_tasks'],
                      per_task_memory_examples=params['per_task_memory_examples'],
                      per_task_examples = params['per_task_examples'],
                      random_class_idx = False)
    label_li = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    n_feature = 28*28

elif params['dataset'] == 'FMNIST':
    benchmark = FashionMNIST(num_tasks=params['num_tasks'],
                             per_task_memory_examples=params['per_task_memory_examples'],
                             per_task_examples = params['per_task_examples'],
                             random_class_idx = False)

    label_li = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 
                  'Ankel boot']
    n_feature = 28*28
    
elif params['dataset'] == 'CIFAR10':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR10(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

elif params['dataset'] == 'CIFAR100':
    label_li = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
    benchmark = CIFAR100(num_tasks=params['num_tasks'],
                        per_task_memory_examples=params['per_task_memory_examples'],
                        per_task_examples = params['per_task_examples'],
                        random_class_idx = False)
    n_feature = 32*32*3

[0 1 2 3 4 5 6 7 8 9]


In [3]:
from algorithms.imbalance import Heuristic2
from metrics import MetricCollector2

backbone = cl.backbones.MLP2Layers(input_dim=784, hidden_dim_1=256, hidden_dim_2=256, output_dim=10)
algorithm = Heuristic2(backbone, benchmark, params, requires_memory=True)
metric_manager_callback = MetricCollector2(num_tasks=params['num_tasks'],
                                                        eval_interval='epoch',
                                                        epochs_per_task=params['epochs_per_task'])

In [4]:
from trainers import ContinualTrainer

trainer = ContinualTrainer(algorithm, params, callbacks=[metric_manager_callback])
trainer.run()
print("final avg-acc", metric_manager_callback.meters['accuracy'].compute_final())
print("final avg-forget", metric_manager_callback.meters['forgetting'].compute_final())

---------------------------- Task 1 -----------------------


solver=<function LS_solver at 0x7ff0422c30d0>
[1] Eval metrics for task 1 >> {'accuracy': 99.71478018520183, 'loss': 5.772249579394399e-05, 'std': 0.020902634181430013}
training_task_end
---------------------------- Task 2 -----------------------
solver=<function LS_solver at 0x7ff0422c30d0>
losses=tensor([[0.0135, 0.0136, 6.6504, 7.1469]])
A_np.shape=(4, 10000)
b_np.shape=(4,)
Elapsed time:141.114
Loss difference:[ 0.06164763  0.09104488  0.09769388 -0.25038531]
len(updated_seq_indices)=9927
[2] Eval metrics for task 1 >> {'accuracy': 93.17809943360604, 'loss': 0.0010321579080946903, 'std': 1.1076148521082407}
[2] Eval metrics for task 2 >> {'accuracy': 95.30767902371633, 'loss': 0.0005400012576556696, 'std': 0.8309348376698111}
training_task_end
---------------------------- Task 3 -----------------------
solver=<function LS_solver at 0x7ff0422c30d0>
losses=tensor([[1.0509e-03, 1.8407e-03, 1.4212e-01, 2.9839e-01, 8.7840e+00, 9.7417e+00]])
A_np.shape=(6, 10000)
b_np.shape=(6,)
Elapsed 

In [5]:
metric_manager_callback.meters['std'].get_std()

[0.020902634181430013,
 1.4465161562131417,
 11.518892745827133,
 12.040199740775613,
 16.296470915993936]

In [6]:
metric_manager_callback.meters['std'].get_data()

array([ 0.021,  1.447, 11.519, 12.04 , 16.296])

In [7]:
metric_manager_callback.meters['accuracy'].get_data()

array([[99.715,  0.   ,  0.   ,  0.   ,  0.   ],
       [93.178, 95.308,  0.   ,  0.   ,  0.   ],
       [88.772, 71.894, 96.469,  0.   ,  0.   ],
       [80.394, 67.999, 79.209, 96.492,  0.   ],
       [77.579, 54.421, 57.831, 78.409, 93.184]])

In [8]:
metric_manager_callback.meters['accuracy'].compute_overall()

86.59553651545876

In [9]:
print(f"accuracy:{np.mean(metric_manager_callback.meters['accuracy'].compute_overall())}")
print(f"fairness:{np.mean(metric_manager_callback.meters['std'].compute_overall())}")


accuracy:86.59553651545876
fairness:8.26459643859825


In [10]:
metric_manager_callback.meters['std'].compute_overall()

8.26459643859825